In [1]:
# import torch 
# from TTS.api import TTS
# from IPython.display import Audio
import yaml
import pandas as pd
import numpy as np
# import torchaudio
# import torchaudio.transforms as T

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# TTS().list_models()

In [48]:
# no good 
# tts = TTS("tts_models/multilingual/multi-dataset/xtts_v1").to(device)

In [49]:
# no good either
# tts2 = TTS("tts_models/multilingual/multi-dataset/xtts_v1.1").to(device)

In [50]:
# pretty good, not as natural sounding as en model 
# tts_es = TTS(model_name='tts_models/es/mai/tacotron2-DDC')

In [51]:
# Super fast 
# tts_fi = TTS(model_name='tts_models/fi/css10/vits').to(device)

In [52]:
# also fast, but produced some weird sounds
# tts_fr = TTS(model_name='tts_models/fr/mai/tacotron2-DDC').to(device)

In [53]:
# Super super fast, very good, but its sr is 16k, not 24k
# tts_it = TTS(model_name='tts_models/it/mai_female/glow-tts').to(device)

In [54]:
# Doesn't work 
# tts_uk = TTS(model_name='tts_models/uk/mai/glow-tts').to(device)

In [55]:
# Doesn't work 
# tts_nl = TTS(model_name = 'tts_models/nl/mai/tacotron2-DDC').to(device)

In [3]:
# good, very fast as well
tts_en= TTS(model_name='tts_models/en/ljspeech/tacotron2-DDC', progress_bar=False).to(device)

 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio P

/scratch/work/lunt1/.conda_envs/tts/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


 > Discriminator Model: hifigan_discriminator
Removing weight norm...


In [2]:
with open("config.yml") as f:
    config = yaml.safe_load(f)

In [3]:
df = pd.read_csv(config["data_args"]["csv_fi"], 
#                  usecols=["sample", "student", "task_id", "cefr_mean", "split", "transcript_normalized", "recording_path"]
                )
# df = df.rename(columns={"transcript_normalized": "text"})

In [58]:
def process(text):
    chunk_size = 10
    for i in range(0, len(text), chunk_size):
        yield " ".join(text[i:i+chunk_size])

In [59]:
df.insert(len(df.columns), "text_chunks", df.text.str.split().apply(lambda l: list(process(l))))

In [42]:
synthesised_sr = 24000
target_sr = 16000

resampler = T.Resample(synthesised_sr, target_sr)

for i, row in df.iloc[regenerate_9].iterrows():
    print(f"--------sample {i} -------")
    whole_utterance = []
    for text_chunk in row.text_chunks:
        text_chunk = text_chunk.replace("ä", "a").replace("ö", "o")
        utterance_chunk = tts_en.tts(text=text_chunk)
        whole_utterance += utterance_chunk
    
    whole_utterance = torch.Tensor(whole_utterance)
    whole_utterance = resampler(whole_utterance)
    
    path = f"./synthesised/synsample_{i}_opiskelija_{row.student}_teht_{row.task_id}.wav"
    torchaudio.save(path, whole_utterance.unsqueeze(dim=0), target_sr)
    print(f">>>>>> sample {i} ready")
    

--------sample 418 -------
 > Text splitted to sentences.
['ei mitaan viela']
   > Decoder stopped with `max_decoder_steps` 10000
 > Processing time: 59.9502739906311
 > Real-time factor: 0.5138475065667724
>>>>>> sample 418 ready
--------sample 533 -------
 > Text splitted to sentences.
['moi ma oon ravintolassa']
   > Decoder stopped with `max_decoder_steps` 10000
 > Processing time: 60.58999752998352
 > Real-time factor: 0.5193307233013561
>>>>>> sample 533 ready
--------sample 959 -------
 > Text splitted to sentences.
['tama on iso perhe talla on isa aiti aa isoaiti']
 > Processing time: 3.570709228515625
 > Real-time factor: 0.5540911671600153
 > Text splitted to sentences.
['tytto ja poika om he tekkevat aa oo launasta oo']
 > Processing time: 2.0000643730163574
 > Real-time factor: 0.4382793312232736
 > Text splitted to sentences.
['he ss he syovat riisia ja pastaa aa he he']
 > Processing time: 2.1993603706359863
 > Real-time factor: 0.4266000718906008
 > Text splitted to sent

In [5]:
def get_failed_synth(path):
    regenerate = []

    with open(path) as file: 
        n = 0
        while True:
            line = file.readline()
            if not line:
                break 
            if line.startswith("------"):
                n = int(line.split()[1])
            if "Decoder stopped with" in line:
                regenerate.append(n)
    return sorted(list(set(regenerate)))

In [20]:
regenerate = get_failed_synth("generate_output.txt")
regenerate_2 = get_failed_synth("generate_output_2.txt")

In [6]:
regenerate_9 = get_failed_synth("generate_output_9.txt")

In [4]:
import os 

# for i, row in df.iloc[regenerate_9].iterrows():
#     path = f"./synthesised/synsample_{i}_opiskelija_{row.student}_teht_{row.task_id}.wav"
#     os.remove(path)

In [6]:
# row = df.iloc[0]
# row.recording_path = "n/a"
# df.append(row)
rows = []
for i, row in df.iterrows():
    path = f"/scratch/work/lunt1/TTS/synthesised/synsample_{i}_opiskelija_{row.student}_teht_{row.task_id}.wav"
    if os.path.exists(path):
        new_row = df.iloc[i]
        new_row["recording_path"] = path
        rows.append(new_row)

/tmp/ipykernel_30027/222950274.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row["recording_path"] = path


In [7]:
df_synth = pd.DataFrame(rows, columns=df.columns)

In [14]:
df_synth = df_synth.drop(columns=["Unnamed: 0"])

In [15]:
df_synth.to_csv("csv_fi_synth.csv")

In [86]:
pd.concat([df, df_synth])

,sample,student,task_id,recording_path,cefr_mean,split,text
0,782,1,1,/m/teamwork/t40511_asr/c/digitala/DigiTala_201...,5,1,jos saisin valita tärkeimmän paikan minulle se...
1,181,3,1,/m/teamwork/t40511_asr/c/digitala/DigiTala_201...,4,1,minä kertoon minun kesämökistä öö se on paras ...
2,913,4,1,/m/teamwork/t40511_asr/c/digitala/DigiTala_201...,5,3,tärkeä paikka minulle on makuuhuoneeni siellä ...
3,1822,5,1,/m/teamwork/t40511_asr/c/digitala/DigiTala_201...,5,2,minulle tärkeä paikka on mun olohuone koska vo...
4,12,7,1,/m/teamwork/t40511_asr/c/digitala/DigiTala_201...,6,1,tämä paikka on minulle tosi tärkeä koska se an...
...,...,...,...,...,...,...,...
2107,588,172,19,/scratch/work/lunt1/TTS/synthesised/synsample_...,5,0,ka hei maija meikäläinen täällä hei öö olin me...
2108,331,303,19,/scratch/work/lunt1/TTS/synthesised/synsample_...,4,0,juu kiitti ku toit mun hupparin kotia nyt on t...
2109,66,270,19,/scratch/work/lunt1/TTS/synthesised/synsample_...,5,2,heippa mä olin huomannu et sä jätit mun huppar...
2110,1010,287,19,/scratch/work/lunt1/TTS/synthesised/synsample_...,6,1,hei maija meikäläinen täällä kiitos tosi paljo...


In [5]:
df

,Unnamed: 0,sample,student,task_id,transcript,recording_path,accuracy_mean,range_mean,fluency_mean,cefr_mean,...,pronunciation_mean,split,transcript_normalized,ASR_transcript,cefr_mean_original,pronunciation_mean_original,fluency_mean_original,accuracy_mean_original,range_mean_original,task_completion_mean_original
0,0,782,1,1,jos saisin valita tärkeimmän paikan minulle se...,/m/teamwork/t40511_asr/c/digitala/DigiTala_201...,3,3,3,5,...,4,1,jos saisin valita tärkeimmän paikan minulle se...,NaN,5.44,3.56,3.32,3.32,2.68,2.96
1,1,181,3,1,minä *kertoon* minun kesämökistä öö<hesitation...,/m/teamwork/t40511_asr/c/digitala/DigiTala_201...,3,2,3,4,...,3,1,minä kertoon minun kesämökistä öö se on paras ...,NaN,3.76,2.84,2.76,2.76,2.04,2.76
2,2,913,4,1,<garbage> tärkeä paikka minulle on makuuhuonee...,/m/teamwork/t40511_asr/c/digitala/DigiTala_201...,4,3,4,5,...,4,3,tärkeä paikka minulle on makuuhuoneeni siellä ...,NaN,5.44,3.84,3.72,3.84,2.56,2.84
3,3,1822,5,1,<bgnoise> minulle tärkeä paikka on mun olohuon...,/m/teamwork/t40511_asr/c/digitala/DigiTala_201...,4,2,4,5,...,4,2,minulle tärkeä paikka on mun olohuone koska vo...,NaN,5.04,3.80,3.64,3.88,2.12,2.48
4,4,12,7,1,<paral> tämä paikka on minulle tosi tärkeä kos...,/m/teamwork/t40511_asr/c/digitala/DigiTala_201...,4,3,3,6,...,4,1,tämä paikka on minulle tosi tärkeä koska se an...,NaN,5.52,3.68,3.36,3.80,2.68,2.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2107,2107,588,172,19,ka<garbage> hei maija meikäläinen<name> täällä...,/m/teamwork/t40511_asr/c/digitala/DigiTala_201...,4,3,3,5,...,4,0,ka hei maija meikäläinen täällä hei öö olin me...,NaN,5.50,4.00,3.50,4.00,3.00,3.00
2108,2108,331,303,19,juu kiitti *ku* toit mun hupparin kotia nyt on...,/m/teamwork/t40511_asr/c/digitala/DigiTala_201...,4,3,4,4,...,4,0,juu kiitti ku toit mun hupparin kotia nyt on t...,NaN,4.50,4.00,4.00,4.00,2.50,3.00
2109,2109,66,270,19,heippa mä olin *huomannu* et sä jätit mun hupp...,/m/teamwork/t40511_asr/c/digitala/DigiTala_201...,4,2,4,5,...,4,2,heippa mä olin huomannu et sä jätit mun huppar...,NaN,5.50,4.00,4.00,4.00,2.50,3.00
2110,2110,1010,287,19,hei maija<name> meikäläinen<name> täällä kiito...,/m/teamwork/t40511_asr/c/digitala/DigiTala_201...,4,3,4,6,...,4,1,hei maija meikäläinen täällä kiitos tosi paljo...,NaN,6.50,4.00,4.00,4.00,3.00,3.00


In [16]:
pd.read_csv("csv_fi_synth.csv")

,Unnamed: 0,sample,student,task_id,transcript,recording_path,accuracy_mean,range_mean,fluency_mean,cefr_mean,...,pronunciation_mean,split,transcript_normalized,ASR_transcript,cefr_mean_original,pronunciation_mean_original,fluency_mean_original,accuracy_mean_original,range_mean_original,task_completion_mean_original
0,0,782,1,1,jos saisin valita tärkeimmän paikan minulle se...,/scratch/work/lunt1/TTS/synthesised/synsample_...,3,3,3,5,...,4,1,jos saisin valita tärkeimmän paikan minulle se...,NaN,5.44,3.56,3.32,3.32,2.68,2.96
1,1,181,3,1,minä *kertoon* minun kesämökistä öö<hesitation...,/scratch/work/lunt1/TTS/synthesised/synsample_...,3,2,3,4,...,3,1,minä kertoon minun kesämökistä öö se on paras ...,NaN,3.76,2.84,2.76,2.76,2.04,2.76
2,2,913,4,1,<garbage> tärkeä paikka minulle on makuuhuonee...,/scratch/work/lunt1/TTS/synthesised/synsample_...,4,3,4,5,...,4,3,tärkeä paikka minulle on makuuhuoneeni siellä ...,NaN,5.44,3.84,3.72,3.84,2.56,2.84
3,3,1822,5,1,<bgnoise> minulle tärkeä paikka on mun olohuon...,/scratch/work/lunt1/TTS/synthesised/synsample_...,4,2,4,5,...,4,2,minulle tärkeä paikka on mun olohuone koska vo...,NaN,5.04,3.80,3.64,3.88,2.12,2.48
4,4,12,7,1,<paral> tämä paikka on minulle tosi tärkeä kos...,/scratch/work/lunt1/TTS/synthesised/synsample_...,4,3,3,6,...,4,1,tämä paikka on minulle tosi tärkeä koska se an...,NaN,5.52,3.68,3.36,3.80,2.68,2.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,2107,588,172,19,ka<garbage> hei maija meikäläinen<name> täällä...,/scratch/work/lunt1/TTS/synthesised/synsample_...,4,3,3,5,...,4,0,ka hei maija meikäläinen täällä hei öö olin me...,NaN,5.50,4.00,3.50,4.00,3.00,3.00
2099,2108,331,303,19,juu kiitti *ku* toit mun hupparin kotia nyt on...,/scratch/work/lunt1/TTS/synthesised/synsample_...,4,3,4,4,...,4,0,juu kiitti ku toit mun hupparin kotia nyt on t...,NaN,4.50,4.00,4.00,4.00,2.50,3.00
2100,2109,66,270,19,heippa mä olin *huomannu* et sä jätit mun hupp...,/scratch/work/lunt1/TTS/synthesised/synsample_...,4,2,4,5,...,4,2,heippa mä olin huomannu et sä jätit mun huppar...,NaN,5.50,4.00,4.00,4.00,2.50,3.00
2101,2110,1010,287,19,hei maija<name> meikäläinen<name> täällä kiito...,/scratch/work/lunt1/TTS/synthesised/synsample_...,4,3,4,6,...,4,1,hei maija meikäläinen täällä kiitos tosi paljo...,NaN,6.50,4.00,4.00,4.00,3.00,3.00


In [13]:
for i in regenerate_9:
    print(f"\item {df.iloc[i].transcript_normalized}")

\item ei mitään vielä
\item moi mä oon ravintolassa
\item tama on iso perhe tällä on isa aiti ää isoäiti tyttö ja poika om he tekkevät ää öö launasta öö he ss he syövät riisiä ja pastaa ää he he sy he juvat vettä ja matoa he ovat ruokapöydällä he aa asuvat aa kerrostalossa öö pyö öö mm is isani öö isan nimi on mark ja äit äidin nimi on tuuli
\item olen hyvä
\item moi ihan hyvä entä sinä
\item moi anna aa hyvä kiitos aa entä sinä
\item onko geenimuunneltu ruoka sinulle aiheena tuttu nii geenimuunneltu on itselleni aihe ja mä varmaan söisin itse geenimuunneltua ruokaa koska must tuntuu että mä en tietäis öö että onko se geenimuunneltu vai ei jos mulle ei siitä erikseen kerrottais öö hyviä puolia geenimuuntelussa on varmaankin se että geenimuuntelussa voi poistaa huonoja asioita ruuassa varmaan ja haluaisin tietää onko tullu minkälaisia seurauksia geenimuuntelussa ruuassa
\item espoosa
\item moikka junassa
